In [1]:
from typing import Dict
import sys
import random

sys.path.append("..")

from src.game.yahtzee_game import YahtzeeGame
from src.simulation.simulator import Simulator
from src.strategies.base_strategy import BaseStrategy
from src.strategies.random_strategy import RandomStrategy
from src.strategies.rule_based_strategy import RuleBasedStrategy
from src.strategies.expected_value_strategy import ExpectedValueStrategy
from src.strategies.expected_value_v2_strategy import ExpectedValueV2Strategy

In [2]:
SEED = 42
random.seed(SEED)

STRATEGIES: Dict[str, BaseStrategy] = {
    "ExpectedV2": ExpectedValueV2Strategy(),
    "Expected": ExpectedValueStrategy(),
    "Rule": RuleBasedStrategy(),
    "Random": RandomStrategy(),
}

Initializing lookup tables for ExpectedValueStrategy...
Computing third roll values...
Computing second roll values...
Computing first roll values...
Lookup tables initialized!


In [3]:
print("Welcome to the Yahtzee Simulator!\n")
game = YahtzeeGame()

for name, strategy in STRATEGIES.items():
    print(f"Adding player: {name} with strategy {strategy.__class__.__name__}")
    game.add_player(name)

# Play the game
print("\nStarting game...\n")

while not game.is_game_over():
    current_player = game.current_player()
    print(f"\nCurrent player: {current_player.name}")

    # First roll is automatic
    dice = game.roll_dice()
    print(f"Roll #1: {dice}")

    # Use strategy for rerolls and scoring
    strategy = STRATEGIES[current_player.name]

    # Second roll
    if game.roll_count < 3:
        dice_to_keep = strategy.select_dice_to_keep(dice, current_player.scorecard)
        kept_indices = ", ".join(str(i + 1) for i in dice_to_keep)
        print(f"Keeping dice at positions: {kept_indices or 'none'}")

        dice = game.roll_dice(dice_to_keep)
        print(f"Roll #2: {dice}")

    # Third roll
    if game.roll_count < 3:
        dice_to_keep = strategy.select_dice_to_keep(dice, current_player.scorecard)
        kept_indices = ", ".join(str(i + 1) for i in dice_to_keep)
        print(f"Keeping dice at positions: {kept_indices or 'none'}")

        dice = game.roll_dice(dice_to_keep)
        print(f"Roll #3: {dice}")

    # Score the roll
    category = strategy.select_category(dice, current_player.scorecard)
    score = current_player.scorecard.calculate_score(category, dice)
    print(f"Scoring {score} in {category.name}")

    game.select_score(category)

# Game over - show results
print("\nGame over!")
for player in game.players:
    print(f"{player.name}: {player.get_total_score()} points")

winner = game.determine_winner()
print(f"\nThe winner is {winner}!")


Welcome to the Yahtzee Simulator!

Adding player: ExpectedV2 with strategy ExpectedValueV2Strategy
Adding player: Expected with strategy ExpectedValueStrategy
Adding player: Rule with strategy RuleBasedStrategy
Adding player: Random with strategy RandomStrategy

Starting game...


Current player: ExpectedV2
Roll #1: [6, 1, 1, 6, 3]
Keeping dice at positions: 1, 4
Roll #2: [6, 2, 2, 6, 2]
Keeping dice at positions: 1, 4
Roll #3: [6, 6, 1, 6, 6]
Scoring 24 in SIXES

Current player: Expected
Roll #1: [6, 5, 1, 5, 4]
Keeping dice at positions: 1, 2, 5
Roll #2: [6, 5, 1, 1, 4]
Keeping dice at positions: 1, 2, 5
Roll #3: [6, 5, 1, 2, 4]
Scoring 18 in CHANCE

Current player: Rule
Roll #1: [2, 5, 5, 1, 5]
Keeping dice at positions: 2, 3, 5
Roll #2: [2, 5, 5, 6, 5]
Keeping dice at positions: 2, 3, 5
Roll #3: [6, 5, 5, 6, 5]
Scoring 25 in FULL_HOUSE

Current player: Random
Roll #1: [5, 4, 2, 4, 5]
Keeping dice at positions: 1, 2
Roll #2: [5, 4, 6, 4, 3]
Keeping dice at positions: 2, 5
Roll #3: [